For Question 1, you are asked to perform the following tasks based on the following target website, which contains artificial content designed 
for this assignment: https://sitescrape.awh.durham.ac.uk/comp42315/ 

a) Please design and implement a solution to crawl the publication title, year and author list of every unique publication record on the target website. [10/35]

b) This information should then be stored and displayed in an appropriate format. Displayed to the user should be: publication title, year, 
#author list, number of authors and impact factor.

c) The records should be manipulatable by the user, at minimum you should be able to display sorted information according to: 
#descending year values, descending number of author values, the titles from A to Z, and finally by the impact of the papers. 

d) Explain your design and highlight any features in this question’s report part of your Jupyter Notebook in no more than 300 words. 
Reflect upon the importance of well structured HTML. [5/35]

In [25]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import VBox, HBox
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model, metrics
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import textstat
import numpy as np
import scipy.stats
import pgmpy as pg
import tabulate as tb
import networkx as nx
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import HillClimbSearch
from pgmpy.metrics.metrics import correlation_score, log_likelihood_score, structure_score
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from pgmpy.estimators import BayesianEstimator
from tqdm import tqdm
from textblob import TextBlob
import os
import textstat
from tqdm import tqdm
import scipy.stats

In [ ]:
#Question (a)
page = requests.get("https://sitescrape.awh.durham.ac.uk/comp42315/publicationfull_year_characteranimation.htm")
page2 = page.content
soup = bs(page2, "html.parser")
divRows = soup.find_all("div", class_="w3-container w3-cell w3-mobile w3-cell-middle")
texts = []
for div in divRows:
    texts.append(div.text.strip())

#Question (b)
publication_titles = []
publisher = []
years = []
authors = []
impact_factor = []
for text in texts:
    content = text.split("\n")
    publication_titles.append(content[0])
    #publisher.append(content[2][0:-6])
    years.append(re.findall(r'\d{4}', content[2])[-1])
    authors.append(content[3])
    if content[4].find("Impact") == -1:
        impact_factor.append("")
    else:
        impact_factor.append(content[4][-6:-1])
#print(publisher)

dict = {'Publication Title': publication_titles, 'Year Published': years, 'Authors': authors, "Impact Factor": impact_factor} #"Publisher": publisher#} 
df1 = pd.DataFrame(dict)
authors = df1["Authors"].values.tolist()  
A = []
for author in authors:
    normalized_authors = author.replace(" and ", ", ")  
    author_list = normalized_authors.split(", ")  
    A.append(len(author_list))
                
df1["Author Count"] = A
display(df1)

#question (c)
items = ['All'] + [col for col in df1.columns if col != 'Authors']
a_slider = widgets.IntSlider(min=1, max=len(df1), step=1, value=5, description="Rows:")
b_select = widgets.Select(options=items, description="Column:")

def view2(x='All', y=3):
    if x == 'All': 
        display(df1.head(y))  
    if x == "Publication Title":
        display(df1.sort_values(by=x, ascending=True).head(y))
    if x == "Year Published":
        display(df1.sort_values(by=x, ascending=False).head(y))
    if x == "Author Count":
        display(df1.sort_values(by=x, ascending=False).head(y))
    if x == "Impact Factor":
        display(df1.sort_values(by=x, ascending=False).head(y))
        
interactive_output = widgets.interactive_output(view2, {'x': b_select, 'y': a_slider})
ui = VBox([HBox([b_select, a_slider]), interactive_output])
display(ui)

The code above is divided into 3 parts. The first involves extracting the html code from the url page and parsing its content. This allows for the relevant information on each publication to be extracted.

The second part is centred around extraction the information required from the data scraped. The year of publication is found using re.findall(r'\d{4}') which finds 4 digit numbers. Because the year published is contained in the same string as the publication source which occasionally contains years in its title, the use of [-1] ensures it is only the last 4 digit number which is appended. The string containing the impact factor is in the format: "impact facotr x.xxx#", hence why the function appends the 5 characters prior to the last one in the string. Because of the “.” in the impact factor the use of re.findall(r'\d{4}') is not possible. Because the number of authors is not explicitly stated on the website crawled, the use of a for loop and string manipulation is required. Finally, there is the possibility of including a column with the name of the journal or book the article was published in (simply remove the # in the for loop). The data is displayed using a pandas dataframe.

Finally, interactive widgets are added which display the information in the dataframe depending on which column the user wishes to engage with. When one of the columns is selected it calls a function which arranges the items in that list in a previously determined order. The output provided by the widgets arranges the rows in alphabetical order for publication title, from newest to oldest for year published, from biggest to smallest impact factor, and from the greatest number of authors to the fewest for author count.

Question 2 (30 marks)

For this question, you will analyse an existing corpus of privacy policies, following the paper https://arxiv.org/abs/2201.08739. You need to download policy-metadata.zip (25.4 MB) and policy-texts.zip (375.1 MB) from https://zenodo.org/records/7426577. You do not need to download any of the labelled datasets or policies, only the raw text and the metadata. The datasets are also provided in the submission folder and uploaded to the Blackboard ULTRA This question consists of 3 parts. [30%]

a) For each policy text file, you need to compute the Flesch Reading Ease score, the SMOG index, the Coleman-Liau index, the Flesch-Kincaid Grade and the Dale-Chall readability score. After manually identifying which of those metrics are given in policy-metadata.csv, you need to compare the metrics you’ve obtained against those provided, identifying any difference. If you have found any difference, reflect on the possible reasons. [10/30]

b) For each policy text file, you need to compute a sentiment analysis, including polarity and subjectivity, and present an overall analysis of the corpus, including a box plot for each metric, as well as as analysis of the median of each metric over the years. [10/30]

c) Finally, you need to do a correlation analysis, assessing if any of the readability metrics in (a) are correlated to the sentiment analysis metrics in (b). [10/30]

[Explain your design and highlight any features in this question’s report part of your Jupyter Notebook in no more than 300 words.]

In [ ]:
files = os.listdir("/home3/COMP42315/policy-texts/policies_fromseg_custom/")
print(len(files))
print(f"Number of files: {len(files)}")
print(files[0:1])
test = open("/home3/COMP42315/policy-texts/policies_fromseg_custom/721ebea55e45486b0fe2e7067d8b3097a588b1f6ad6a442ccc31af7b.txt");

file_contents = test.read()
print(file_contents[0:100]);
test.close();

In [ ]:
metadata = pd.read_csv("/home3/COMP42315/policy-metadata/policy-metadata.csv")  

In [ ]:
folder_path = "/home3/COMP42315/policy-texts/policies_fromseg_custom/"

files = os.listdir(folder_path)

readability_scores = []


for file_name in tqdm(files, desc="Processing Files", unit="file"):
    try:

        file_path = os.path.join(folder_path, file_name)
        
        
        with open(file_path, 'r', encoding='utf-8') as test:
            file_contents = test.read()

        
        scores = [
            textstat.flesch_reading_ease(file_contents),
            textstat.flesch_kincaid_grade(file_contents),
            textstat.smog_index(file_contents),
            textstat.coleman_liau_index(file_contents),
            textstat.dale_chall_readability_score(file_contents)
        ]
        
        
        readability_scores.append(scores)

    except Exception as e:
        print(f"Error reading {file_name}: {e}")


df_readability = pd.DataFrame(readability_scores, columns=[
    'Flesch Reading Ease', 
    'Flesch-Kincaid Grade', 
    'SMOG Index', 
    'Coleman-Liau Index', 
    'Dale-Chall Readability Score'
])


df_readability['File Name'] = files


print(df_readability.head())

In [ ]:
metadata_subset = metadata[["flesch", "smog", "cl"]] 
df_subset = df_readability[['Flesch Reading Ease', 'SMOG Index', 'Coleman-Liau Index']]
comparison_data = pd.concat([df_subset, metadata_subset], axis=1)
comparison_data.describe()

#the three common features have a higher mean in the policy-metada file than the averages calculated using the text files. However, the differences
#are extremly small (both in absolute and relative terms). These small differneces may be because of the much greater number of measurements inlcuded
#in the policy-metadata file (645,125 vs 56,415). 

In [ ]:
#Question 2(b)
folder_path = "/home3/COMP42315/policy-texts/policies_fromseg_custom/"
files = os.listdir(folder_path)


cleaned_texts = []


for file_name in tqdm(files, desc="Processing Files", unit="file"):
    try:
        file_path = os.path.join(folder_path, file_name)
        

        with open(file_path, 'r', encoding='utf-8') as test:
            file_contents = test.read()


        cleaned_text = re.sub(r'[^a-zA-Z\s]', '', file_contents).lower()

        
        cleaned_texts.append(cleaned_text)

    except Exception as e:
        print(f"Error processing {file_name}: {e}")



In [ ]:
def sentiment_analysis(cleaned_texts):
    df2 = pd.DataFrame()
    polarity_values = []
    subjectivity_values = []
    

    for text in tqdm(cleaned_texts, desc="Analyzing Sentiment", unit="text"):
        text_blob = TextBlob(text)
        polarity_values.append(text_blob.sentiment.polarity) 
        subjectivity_values.append(text_blob.sentiment.subjectivity)
    
    df2["polarity"] = polarity_values
    df2["subjectivity"] = subjectivity_values
    return df2


df2 = sentiment_analysis(cleaned_texts)

In [ ]:
flesch_over_time = metadata['flesch'].groupby(metadata['year']) 
F = flesch_over_time.describe()
F = F.reset_index()
x = F["year"]
y = F["50%"]
plt.scatter(x, y, marker="*", s=10)
plt.plot(x, y)
#The flesch reading score graph shows that after increasing in the first four years, the average score has decreased over time. The scire for the first
# is an outlier and is substantially lower than any of the other scores. Excluding the first measurement, the overall decrease has been from around 38
#in 2000 to approximately 32 in 2021. 

In [ ]:
smog_over_time = metadata['smog'].groupby(metadata['year'])
S = smog_over_time.describe().reset_index()
x=S["year"]
y=S["50%"]
plt.scatter(x, y, marker="*", s=10)
plt.plot(x, y)
#the graph for SMOG index scores shows that after an intial decrease in the first three years, the median score has increased steadily over time.
#As with the Flesch Reading graph, the difference between the first two measurements is stark, alathough in the general context the first measurement
#is not an outlier. The change overtime (from a low of around 15 to a high of nearly 16.75 in 2020) is smaller than the equivalent for the 
#Flesch Reading Score. 

In [ ]:
cl_over_time = metadata['cl'].groupby(metadata['year'])
CL = cl_over_time.describe().reset_index()
x=CL["year"]
y=CL["50%"]
plt.scatter(x, y, marker="*", s=10)
plt.plot(x, y)
#THe scatter plot for the Coleman-Liau Index shows a gradual upwards trend over tiem. As with both previous graphs the difference between the
#first two measurements is stark, with the median for 1996 being an outlier. From 1999, the difference in the median score between years is low. The
#Colemian-Liau Index shows the least change over time of the three graph, rising from a low of approximately 12.75 in 1998 to a high of around
#13.3 in 2021. 

In [ ]:
#polarity boxplot
plt.figure(figsize=(12, 6))  
plt.boxplot([df2['polarity'], df2['subjectivity']], 
            tick_labels=["Polarity Scores", "Subjectivity Scores"], 
            widths=0.2)
plt.title('Polarity Boxplot')
plt.ylabel('Values')
plt.tight_layout()
#plt.show()

#The graph below presents the boxplots for polarity and subjectivity. Both have a significant number of outlier measurements.


In [ ]:
plt.figure(figsize=(12, 6))
plt.boxplot([df_readability['Dale-Chall Readability Score'], df_readability['Coleman-Liau Index'], df_readability['SMOG Index']], 
            tick_labels=["Dale-Chall Readability Score", "Coleman_Liau Index", "SMOG Index"]) 
plt.axis([None, None, -5, 40])
plt.title('Comparison of Various Readability Metrics')
plt.ylabel('Values')
plt.tight_layout()
plt.show()
#the boxplot below shows the boxplots for the Dale-Chale Readability Score, the Coleman-Liau Index and the SMOG Index. All three have a significant
#number of outliers. 

In [ ]:
plt.figure(figsize=(12, 6))  
plt.boxplot([df_readability['Flesch Reading Ease'], df_readability['Flesch-Kincaid Grade']], 
            tick_labels=["Flesch Reading Ease", "Flesch-Kincaid Grade"], 
            widths=0.2)
plt.title('Comparison of Various Metrics')
plt.axis([None, None, -75, 75])
plt.ylabel('Values')
plt.title('Flesch Reading Ease Boxplot')
plt.ylabel('Values')
plt.tight_layout()
plt.show()
#The graph below shows the boxplots for the Flesch Reading Ease score and the Flesch_kincaid Grade. Both have a significant number of outliers. 

In [ ]:
#Question (c)
horizontal_concat_clean = pd.concat([df_readability, df2], axis=1)
horizontal_concat.corr()

In [ ]:
print("Polarity Values")
for i in range(5):
    predictors = horizontal_concat_clean.iloc[:, i]
    predictors = pd.to_numeric(predictors, errors='coerce')
    polarity = pd.to_numeric(horizontal_concat_clean["polarity"], errors='coerce')
    correlation, p_value = scipy.stats.pearsonr(predictors, polarity)
    print(f"Feature {horizontal_concat_clean.columns[i]}: Pearson correlation = {correlation}, p-value = {p_value}")

print("Subjectivity Values")
for i in range(5):
    predictors = horizontal_concat_clean.iloc[:, i]
    predictors = pd.to_numeric(predictors, errors='coerce')
    subjectivity = pd.to_numeric(horizontal_concat_clean["subjectivity"], errors='coerce')
    correlation, p_value = scipy.stats.pearsonr(predictors, subjectivity)
    print(f"Feature {horizontal_concat_clean.columns[i]}: Pearson correlation = {correlation}, p-value = {p_value}")


The three readability scores contained in the policy-metadata file were: Flesch Reading Ease, SMOG Index and the Coleman-Liau Index. After creating a subset of the original metadata file with only these three measurements, the subset is merged horizontally with a data frame which contains the values calculated using the readability function.

In order to ease the process of calculating subjectivity and polarity for the text files, the function re is used to remove any non-alphabetical characters from the text files. Once this is done the remaining letters are all converted to lower case. The textblob package is then used to calculate each individual polarity and subjectivity score.

Only three of the readability metrics are presented in the graphs showing a change in the median score over time. This is because they are the three that appear in the metadata file and therefore can be analysed by year. The graphs are presented separately as they have different scales.
Where possible, the boxplots are plotted on the same graph. However, this is not possible for all of the different measures due to differences in scale. Note that for boxplots 2 and 3 the y axis has been limited to exclude a small number of extreme outliers. This is because these outliers stretch the boxplots to an extent where they are almost no longer visible. These outliers can be viewed by removing the plt.axis() command in the code.

Finally, a correlation matrix is plotted to visualise the correlation between the readability metrics and polarity and sensitivity. Using Pearson’s coefficient, two for loops are used to see how many of these correlations are statistically significant. Whilst all correlations are found to be statistically significant, none of the variables show strong correlations with either polarity or sensitivity (the strongest correlation is -0.2).

# Question 3 (35 marks)

For this question, you are asked to perform the task based on the target dataset (drone.csv), which you can download separately on Blackboard Ultra. The target drone dataset contains different features related to the drone physical attributes.  The target variable is 'warning', i.e. warning level. If the warning  level is high then a drone must be grounded immediately to avoid any damage to the drone itself as well as the surroundings (in case of a crash). Design and implement the solution to use data analysis and visualisation for the following tasks: [35%]

a) You are required to extract a subset that includes the 'defined_features' and the 'target_variable' (in the subset, there will be 11 features in total including target variable). You are required to extract the 'defined_features' that are as indicated below: 

defined_features = ['voltage_v', 'voltage_filtered_v', 'current_a', 'current_filtered_a', 'discharged_mah', 'remaining', 'scale', 'load', 'ram_usage'] 

Perform exploratory data analysis on the drone dataset ('defined_features'), highlight the features that are statistically important and highly correlated to the 'target variable', and visualise them legibly using an appropriate visual method. Save the statistically important features as a subset 'selected_features' and compare them with the 'defined_features'. Highlight any differences and report your findings. [10/35]

b) Perform analysis to identify the complex relationship between drone physical attributes ('selected_features') and the warning level ('target_variable') using a probabilistic method. Highlight and visualise the attributes with the highest probabilistic relationship with the target variable. Justify the design choice and showcase the findings using an appropriate visualisation tool. [18/35]

c) Perform predictive analysis using at least one machine learning algorithms and validate its performance based on suitable performance metrics. Justify the design choice and showcase the findings using an appropriate visualisation tool. [7/35]

[Explain your design and highlight any features in this question's report part of your Jupyter Notebook in no more than 400 words.]

In [ ]:
#question 3(a)
df = pd.read_csv("drone.csv")
defined_features = df.drop("timestamp",axis=1)
defined_features.describe()
#display(defined_features)

correlation_matrix = defined_features.corr()
#display(correlation_matrix)

na_counts = df.isna().sum()
#print(na_counts)
defined_features.dropna()

target_variable = df[["warning"]]

selected_features_list = []
variable = []
Correlation = []
P_value = []

for i in range(9):
    feature = defined_features.iloc[:, i]
    correlation, p_value = scipy.stats.pearsonr(feature, defined_features["warning"])
    print(f"Feature {defined_features.columns[i]}: Pearson correlation = {correlation}, p-value = {p_value}")
    if p_value < 0.05:
        selected_features_list.append(defined_features.columns[i])
        variable.append(defined_features.columns[i])
        Correlation.append(correlation)
        P_value.append(p_value)

selected_features = defined_features[selected_features_list]

In [ ]:
sns.pairplot(df, 
             x_vars=['voltage_v', 'voltage_filtered_v', 'current_a', 'current_filtered_a', 
                     'remaining', 'discharged_mah', 'scale'],
             y_vars=['warning'], 
             height=2.5) 
plt.suptitle("Pairplot of Defined Features and Target Variable", y=1.02)
plt.tight_layout()
plt.show()
#the plots below illustrate the relationship between the selected_features and warning. 

In [ ]:
correlation_results = pd.DataFrame(list(zip(variable, Correlation, P_value)), columns = ["Variable", "Correlation", "p_value"])
correlation_results

#a total of 7 variables are included in selected_variables and are listed below. The variables load and ram_usage
#are the only two varaibles in the defined_features dataset that are not found to be correlated with warning.

In [ ]:
#question 3(b)
#in order to use a bayesian model it is necessary to discretise the variables. #A Bayesian network is chosen to model the probablistic relationship 
#as it allows for the modelling of complex dependencies between multiple variables whilst accounting for uncertanties and prioir knowledge. 

voltage_v_labels = ['1. less than 16', '2. Between: 16 and 18', '3. Between: 18 and 20', '4. Greater than: 20']
voltage_filtered_v_labels = ['1. less than 16', '2. Between: 16 and 18', '3. Between: 18 and 20', '4. Greater than: 20']
current_a_labels = ['1. less than 16', '2. Between: 16 and 18', '3. Between: 18 and 20', '4. Greater than: 20']
current_filtered_a_labels = ['1. less than 16', '2. Between: 16 and 18', '3. Between: 18 and 20', '4. Greater than: 20']
remaining_labels = ["1. 0", "2. (0, 0.5)", "3. >0.5"]
scale_labels = ['2. Less than 1.157143', '1. Exactly 1.157143']
warning_labels = ['1. 0', '2. 1', '3. 2', '4. 3']

target_variable = pd.DataFrame(defined_features.warning)
target_variable['warning'] = pd.cut(target_variable['warning'], bins=4, labels=warning_labels, precision=2)
for column in target_variable:
    print(target_variable.groupby(column)[column].count().reset_index(name='Count').to_dict(orient='records'))

def make_discrete(df):

    discrete_df = pd.DataFrame()

    discrete_df['voltage_v'] = pd.cut(df['voltage_v'], bins=4, labels=voltage_v_labels, precision=2)
    discrete_df['voltage_filtered_v'] = pd.cut(df['voltage_filtered_v'], bins=4, labels=voltage_filtered_v_labels, precision=2)
    discrete_df['current_a'] = pd.cut(df['current_a'], bins=4, labels=current_a_labels, precision=2)
    discrete_df['current_filtered_a'] = pd.cut(df['current_filtered_a'], bins=4, labels=current_filtered_a_labels, precision=2)
    discrete_df['remaining'] = pd.cut(df['remaining'], bins=3, labels=remaining_labels, precision=2)
    discrete_df['scale'] = pd.cut(df['scale'], bins=2, labels=scale_labels, precision=2)
    discrete_df = discrete_df.astype('object')
    
    return discrete_df

discrete_df = make_discrete(selected_features) 
discrete_df = pd.concat([discrete_df, target_variable], axis=1)
for column in discrete_df:
    discrete_df.groupby(column)[column].count().reset_index(name='Count').to_dict(orient='records')

#splitting data into training and testing sets. 
training_data, testing_data = train_test_split(discrete_df, test_size=0.2, random_state=20)

#fit Bayesian Network
hc = HillClimbSearch(data=training_data)
estimate = hc.estimate(scoring_method='k2score')
model = BayesianNetwork(estimate)

model.cpds = []

model.fit(data=training_data,
    estimator=BayesianEstimator,
    prior_type='BDeu',
    equivalent_sample_size=10)

In [ ]:
plt.figure(3,figsize=(14,10))
G = nx.DiGraph()
G.add_edges_from(model.edges)


G.add_nodes_from(model.nodes)
pos = nx.circular_layout(G)
DAG = G.to_directed()
nx.topological_sort(DAG)

nx.draw_networkx(G,
                pos=pos,
                with_labels=True,
                node_size=[2000] * len(G.nodes),
                arrowsize=30,
                alpha=0.7,
                font_weight="bold",
                width=2.0) 

tt_g = G.subgraph(nodes=['resourceState'])
nx.draw(tt_g, pos=pos, with_labels=False, arrowsize=0, node_size=4100, alpha=0.7, font_weight="bold", node_color='#063970')

plt.show()
#graph below shows clearly the different conditional relationships amongst all the variables selected. The variables
#with the highest probablistic relationship with the target variable are scale, remaining, voltage_filtered_v and 
#current_filtered_a.

In [ ]:
nodes_to_include = ['scale', 'remaining', 'voltage_filtered_v', 'current_filtered_a', 'warning']  

subgraph = G.subgraph(nodes=nodes_to_include)
pos = nx.circular_layout(subgraph)  # You can also experiment with other layouts (e.g., spring_layout)
nx.draw_networkx(subgraph,
                 pos=pos,
                 with_labels=True,
                 node_size=[2000] * len(subgraph.nodes),
                 arrowsize=30,
                 alpha=0.7,
                 font_weight="bold",
                 width=2.0)
plt.show()
#network below only shows those nodes which have the highest probablistic relationship with warning.

In [ ]:
#determining accuracy, precision and F1 score for each variable in the bayesian network
accuracy_dict = {}
f1_dict = {}
precision_dict = {}

for column in testing_data:
    predict_data = testing_data.copy()
    predict_data.drop(column, axis=1, inplace=True)
    y_pred = model.predict(predict_data)
    accuracy = accuracy_score(testing_data[column], y_pred)
    print(f'{column} Accuracy score: {accuracy}')
    accuracy_dict[column] = accuracy
    f1 = f1_score(testing_data[column], y_pred, average='weighted')  # 'weighted' takes class imbalance into account
    print(f'{column} F1 score: {f1}')
    f1_dict[column] = f1
    precision = precision_score(testing_data[column], y_pred, average='weighted')  # 'weighted' for class imbalance
    print(f'{column} Precision score: {precision}')
    precision_dict[column] = precision

In [ ]:
#question 3(c)

#Binomial logistic regression is used. This classification method is easier to interpret, train and implement than
#a linear regression model. In the case of warning where the principal concern is distinguishing between 0 and 3 
#logistic regression is a suitable method of anlaysis. 
df = df[df["warning"].isin([0, 3])]
y = df["warning"]
X = df[["scale", "voltage_filtered_v", "remaining", "current_filtered_a"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

reg = linear_model.LogisticRegression(max_iter=10000, random_state=0)
reg.fit(X_train, y_train)

#The results of the logistic regression are visualised through the use of a confusion matrix which shows the 
#distribution of the model's predictions, and whether they were correct. 
y_pred = reg.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cm_display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=reg.classes_)
cm_display.plot(cmap='Blues', values_format='d', ax=None)
plt.title('Confusion Matrix')
plt.show()

print(f"Logistic Regression model accuracy: {metrics.accuracy_score(y_test, y_pred) * 100:.2f}%")
print(metrics.classification_report(y_test, y_pred))
print(f"MSE: {mean_squared_error(y_test, y_pred)}")
print(f"r-squared score: {r2_score(y_test, y_pred)}")

The first part of the answer deals with finding which variables are correlated with the target variable. The method above uses pearson's r to extract the statistically significant variables from defined_features. The for loop finds the statistically significant variables and then appends them to an empty list. This list is then used to create the selected_features subset.

The selected features subset is created using <if p_value < 0.05> statement. The results show that 7 variables are correlated with warning, with only load and ram_usage not found to be correlated. When conducting statistical analysis, the discharged_mah value is excluded. This is because its correlation is far weaker than the other variables and is therefore unlikely to significantly improve any future model. This weaker correlation is visible in the pairplots generated. This approach of excluding discharged_mah is vindicated by the high accuracy scores found at the end of the answer.

A Bayesian network is then used to analyse the conditional dependencies between the different variables. The model is trained on 80% of the data from the discretised data frame. During discretisation each variable was broken down into multiple categories, with the cut-off points chosen to best reflect the structure of each variable. The fitted Bayesian model is then evaluated using the K2 score. The model's accuracy scores were high across the board, with only the accuracy score for voltage_v falling below 90%.

When conducting the logistic regression, the model is limited to predicting whether warning will equal 0 or 3, excluding levels 1 and 2. This is because the data is not uniformly distributed; levels 0 and 3 contain over 99% of all the measurements. The main aim of the model is therefore to distinguish between the two main categories, which it does with a high level of accuracy and precision. The predictor variables selected are those that the Bayesian network identified as having a conditional probability relationship with warning.

When analysing the statistics printed the model does exceptionally well across the board, with 99% of the model's predictions being correct. The results of the classification report show that the model does slightly better in predicting warning level 3 compared to level 0 although the difference is minor. The r-squared score of nearly 90%, meaning that the model's fit is extremely high. This validates the decision to only include the 4 predictor variables chosen.